### Pycrash impact simulation using impulse-momentum with vehicle motion

In [ ]:
# %% allow reloading of modules
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.getcwd()

##### If running Pycrash outside of Python, add location to path:

In [ ]:
import sys
sys.path.insert(0,'path to pycrash')

### Import modules

In [ ]:
from pycrash.impact_main import Impact
from pycrash.vehicle import Vehicle

In [ ]:
import pandas as pd
import numpy as np
from scipy import integrate
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import math

In [ ]:
# python dictionary containing vehicle specifications
import projects.data.vehicle_data_collection as vehData

In [ ]:
from pycrash.visualization.kinematics_compare import compare_kinematics
from pycrash.visualization.cg_motion_compare import cg_motion

### Create Vehicles

In [ ]:
veh1 = Vehicle('Striking', vehData.vehicle_data['ChevroletMalibu2004'])
veh1.striking = True  # <- set to striking vehicle

veh2 = Vehicle('Struck', vehData.vehicle_data['HondaAccord'])
veh2.striking = False  # <- set to struck

# create list of impact object
vehicles = [veh1, veh2]

#### Vehicles inputs

In [ ]:
t = [0, 5]
brake = [0, 0]
throttle = [0, 0]
steer = [0, 0]
veh1.time_inputs(t, throttle, brake, steer, show_plot=False)
veh1.vx_initial = 30

In [ ]:
veh2.time_inputs(t, throttle, brake, steer, show_plot=False)
veh2.vx_initial = 0

### Create impact object

#### Manually define impact points

In [ ]:
# vehicle 1
# impact point = (x, y, impact plane normal angle [deg])
veh1.impact_points = [(veh1.lcgf + veh1.f_hang - 1, (-1 * veh1.width / 2) + 1, -16.5)]
#veh1.impact_points = [(-veh1.lcgr + 2, -veh1.width / 2, 90), (veh1.lcgf + veh1.f_hang, veh1.width / 2, 0)] # right front corner

# vehicle 2
veh2.edgeimpact = 3
veh2.edgeimpact_points = [(-1 * veh2.lcgr - veh2.r_hang, -1 * veh2.width / 2, veh2.lcgf + veh2.f_hang, -1 * veh2.width / 2)]

### Initial Positions

In [ ]:
# Vehicle 1
veh1.init_x_pos = 0
veh1.init_y_pos = 0
veh1.head_angle = 0

# Vehicle 2
veh2.init_x_pos = 40
veh2.init_y_pos = -10
veh2.head_angle = -90

### Parametric Analysis

In [ ]:
# inputs kept constant
t = [0, 1, 2, 3, 4, 5]
throttle = [0] * len(t)
# inputs to vary
# brake - apply brake and hold
t_brake = 2       # <- time to apply brake
max_brake = 1      # <- maximum brake applied
brake = [0] * len(t)
for i in range(0, len(t)):
    if t[i] >= t_brake:
        """ insert random generator 0 to 1"""
        brake[i:] = random * max_brake  # apply constant braking to the end
        break
        
""" same for steering """
    


steer = [0, 0]
veh1.time_inputs(t, throttle, brake, steer, show_plot=False)
veh1.vx_initial = 30

In [ ]:
# name, endTime, impact_type, vehicle_list, impact_order=None, impc_inputs=None, user_sim_defaults=None
imp = Impact('Scenario1', 2, 'IMPC', [veh1, veh2])

#### Show Initial Positions

In [ ]:
imp.show_initial_position()

### Run simulation

In [ ]:
imp.simulate()

In [ ]:
imp.plot_impact(0)

In [ ]:
imp.plot_vehicle_motion(5, show_vector=True)

### Load PC-Crash Data

In [ ]:
pc_crash_column_names = ['t', 'ax', 'ay', 'az', 'phi_deg', 'lf_fy', 'rf_fy',
                         'lr_fy', 'rr_fy', 'delta_deg', 'rf_delta_deg', 'steer',
                         'steer_rate', 'X', 'Y', 'Z', 'roll', 'pitch', 'theta_deg',
                         'Vx', 'Vy', 'Vz', 'rf_fz', 'lf_fz', 'rr_fz', 'lr_fz',
                         'rf_alpha', 'lf_alpha', 'lr_alpha', 'rr_alpha']

In [ ]:
test_file_list = os.listdir(os.path.join(os.getcwd(), 'data', 'input'))
print('List of tests for analysis:')
test_file_list

In [ ]:
test_do = 4 # <- cho|ose test number from list to process
print(f'Test to be processed: {test_file_list[test_do]}')

In [ ]:
df = pd.read_excel(os.path.join(os.getcwd(), 'data', 'input', test_file_list[test_do]),
                            na_filter = False, header = None, names = pc_crash_column_names, skiprows = 2,
                            usecols = 'A:AD', nrows=51, sheet_name='target data')

In [ ]:
df = pd.read_excel(os.path.join(os.getcwd(), 'data', 'input', test_file_list[test_do]),
                            na_filter = False, header = None, names = pc_crash_column_names, skiprows = 2,
                            usecols = 'A:AD', nrows=51, sheet_name='bullet data')

In [ ]:
df.head()

In [ ]:
#df.steer = [x * -1 for x in df.steer]  # reverse steer - PC-Crash is positive ccw

# convert velocities to fps
df.Vx = [x * 1.46667 for x in df.Vx]
df.Vy = [x * -1.46667 for x in df.Vy]
df.Vz = [x * 1.46667 for x in df.Vz]

# convert acceleration to fps/s
df.ax = [x * 32.2 for x in df.ax]
df.ay = [x * -32.2 for x in df.ay]
df.az = [x * 32.2 for x in df.az]

# convert tire forces to lb
df.lf_fy = [x * 1000 for x in df.lf_fy]
df.rf_fy = [x * 1000 for x in df.rf_fy]
df.lr_fy = [x * 1000 for x in df.lr_fy]
df.rr_fy = [x * 1000 for x in df.rr_fy]

# steer angle in radians
df['delta_rad'] = [x / 180 * np.pi for x in df.delta_deg]

# integrate velocities to get displacements
df['Dx'] = df.X
df['Dy'] = [x * -1 for x in df.Y]

df['theta_deg'] = [x * -1 for x in df.theta_deg]
df.head()

In [ ]:
target = df.copy()

In [ ]:
bullet = df.copy()

In [ ]:
# calculate vehicle slip angle for pycrash model - need to correct
for j in range(0, len(imp.vehicles)):
    phi_rad = []
    phi_deg = []
    for i in range(len(imp.vehicles[j].model.t)):
        phi_rad.append(math.atan2(imp.vehicles[j].model.vy[i], imp.vehicles[j].model.vx[i]))
        phi_deg.append(math.atan2(imp.vehicles[j].model.vy[i], imp.vehicles[j].model.vx[i])*(180 / math.pi))
    imp.vehicles[j].model['phi_rad'] = phi_rad
    imp.vehicles[j].model['phi_deg'] = phi_deg

In [ ]:
compare_kinematics(imp.vehicles[1].model, target, 'Pycrash', 'PC-Crash')

In [ ]:
cg_motion(imp.vehicles[1].model, target, 'Pycrash', 'PC-Crash')